<a href="https://colab.research.google.com/github/shanikairoshi/QML-and-Deep-Unfolded_QML/blob/main/Classical_SVM_vs_Deep_unfolded_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss

# Load dataset (e.g., Iris)
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binary classification for simplicity (setosa vs. non-setosa)
y = (y == 0).astype(int)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Classical SVM using scikit-learn
svm_classical = SVC(kernel='linear', probability=True)
svm_classical.fit(X_train, y_train)

# Predict on test data
y_pred = svm_classical.predict(X_test)
y_prob = svm_classical.predict_proba(X_test)

# Compute accuracy and loss
accuracy_classical = accuracy_score(y_test, y_pred)
loss_classical = log_loss(y_test, y_prob)

print(f"Classical SVM Accuracy: {accuracy_classical}")
print(f"Classical SVM Loss: {loss_classical}")


Classical SVM Accuracy: 1.0
Classical SVM Loss: 0.020701257697304402


In [8]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Define the deep unfolded SVM with iterative optimization
class DeepUnfoldingSVM(torch.nn.Module):
    def __init__(self, input_size):
        super(DeepUnfoldingSVM, self).__init__()
        self.weights = torch.nn.Parameter(torch.zeros(input_size))
        self.bias = torch.nn.Parameter(torch.zeros(1))
        self.lr = 0.1  # Initial learning rate

    def forward(self, x):
        return torch.sign(torch.matmul(x, self.weights) + self.bias)

    def loss_fn(self, y_pred, y_true):
      hinge_loss = torch.mean(torch.clamp(1 - y_true * y_pred, min=0))
      l2_reg = torch.sum(self.weights ** 2)  # L2 regularization term
      return hinge_loss + 0.01 * l2_reg  # 0.01 is the regularization coefficient
# Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binary classification for simplicity (setosa vs. non-setosa)
y = (y == 0).astype(int)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Convert data to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Initialize model and optimizer
model = DeepUnfoldingSVM(input_size=X_train.shape[1])
optimizer = torch.optim.Adam([model.weights, model.bias], lr=0.01)


# Training loop
epochs = 500
batch_size = 32  # Example mini-batch size
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = model.loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

# Evaluation on test set
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    y_pred_test = y_pred_test.numpy()
    y_pred_test = (y_pred_test > 0).astype(int)

# Compute accuracy and loss
accuracy_unfolded = accuracy_score(y_test, y_pred_test)
loss_unfolded = log_loss(y_test, y_pred_test)

print(f"Deep Unfolded SVM Accuracy: {accuracy_unfolded}")
print(f"Deep Unfolded SVM Loss: {loss_unfolded}")


Deep Unfolded SVM Accuracy: 0.5777777777777777
Deep Unfolded SVM Loss: 15.218431430960578
